In [44]:
import pandas as pd
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()

RAW_SAMPLE_1 = os.getenv("RAW_SAMPLE_1")
df_read = pd.read_excel(RAW_SAMPLE_1, sheet_name=0)

df = df_read.copy()

df.head()

In [ ]:
# Select relevant columns, make new columns, and drop NaN rows
df = df[['Well', 'Cq']]

df['Sample'] = "sample"
df['mtDNA1'] = "mtDNA1"
df['mtDNA2'] = "mtDNA2"

df = df.loc[:,["Well", "Sample", "mtDNA1", "mtDNA2", "Cq"]]

df = df.dropna()

df

In [ ]:
RAW_SAMPLE_1_DIAGRAM = os.getenv("RAW_SAMPLE_1_DIAGRAM")
df_diagram_read = pd.read_excel(RAW_SAMPLE_1_DIAGRAM)

df_diagram = df_diagram_read.copy()

df_diagram.head()